# Generación de puntos aleatorios

Haga un programa que genere al azar estados del dipolo $\vec{u}$ uniformemente distribuidos sobre la esfera. Esto se logra de manera muy sencilla en coordenadas cilíndricas $(r, \theta, z)$, fijando $r=u$ y escogiendo $\theta$ al azar uniformemente distribuído entre $\theta \in(0,2 \pi)$ y $z$ uniformemente distribuido entre $z \in(-u, u)$. Por lo tanto, basta con generar $u_{z}$ uniformemente distribuido en el intervalo $u_{z} \in(-u, u)$. Asuma $\|\vec{u}\|=2 \mathrm{y}$ $B=10$ y tome un número de muestras $N=10000$ muestras para cada temperatura (es decir, para cada valor de $\mathrm{x}$ ).



In [ ]:
# Utilizando random para genera números aleatorios.
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.constants as constants

#Definción del Método de Monte Carlo

In [ ]:
#@title Definición de la funciónes a usar

#Monte Carlo para el calculo del area de un circulo
def mc_pi_aprox_circ(N=10000):
    plt.figure(figsize=(8,8))  # tamaño de la figura
    x, y = np.random.uniform(-1, 1, size=(2, N))
    interior = (x**2 + y**2) <= 1 #Lugar donde se pone la función
    # pi(r^2)/(L^2)=Puntos Circ/Puntos Cuadrado
    pi = interior.sum() * 4 / N #(Puntos Circ/Puntos Cuadrado)*(L^2)/(r^2)
    error = abs((pi - np.pi) / pi) * 100
    exterior = np.invert(interior)
    plt.plot(x[interior], y[interior], 'b.')
    plt.plot(x[exterior], y[exterior], 'r.')
    plt.plot(0, 0, label='$\hat \pi$ = {:4.4f}\nerror = {:4.4f}%'
             .format(pi,error), alpha=0)
    plt.axis('square')
    plt.legend(frameon=True, framealpha=0.9, fontsize=16)

#Monte Carlo para el calculo del area de una función
def mc_area_aprox(fn,a,b,N=10000):
    x = np.random.uniform(a, b, size=(1, N))#Revisar dimensionalidad
    y = np.random.uniform(0,fn(x).max(), size=(1, N))
    #Puntos bajo la curva
    interior = y <= fn(x)
    #Área debajo la funcion/Área del cuadrado =
    #Puntos debajo de la funcion/Numero de puntos totales
    area = (interior.sum() * (b-a)*fn(x).max()) / N#Área debajo la funcion =
    #(Puntos debajo de la funcion*Área del cuadrado)/Numero de puntos totales)
    area_ex = integrate.quad(fn,a,b)[0]
    error = abs((area - area_ex) / area) * 100
    return area, error

mc_area_aprox = np.vectorize(mc_area_aprox)

#Monte Carlo para el calculo del area de una función y su grafica
def mc_area_graph_aprox(fn,a,b,N=10000):
    plt.figure(figsize=(8,8))  # tamaño de la figura
    x = np.random.uniform(a, b, size=(1, N))#Revisar dimensionalidad
    y = np.random.uniform(0,fn(x).max(), size=(1, N))
    #Puntos bajo la curva
    interior = y <= fn(x)
    area = (interior.sum() * (b-a)*fn(x).max()) / N
    area_ex = integrate.quad(fn,a,b)[0]
    error = abs((area - area_ex) / area) * 100
    # area, error = mc_area_aprox(fn,a,b,N=10000)
    exterior = np.invert(interior)
    plt.xlim(a, b)
    plt.ylim(0, fn(x).max())
    plt.plot(x[interior], y[interior], 'b.')
    plt.plot(x[exterior], y[exterior], 'r.')
    plt.plot(0, 0, label='$area:$ = {:4.4f}\nerror = {:4.4f}%'
             .format(area,error), alpha=0)
    # plt.axis('square')
    plt.legend(frameon=True, framealpha=0.9, fontsize=16)

In [ ]:
#@title Circulo
mc_pi_aprox_circ(int(1e4))

In [ ]:
#@title e^x
a,b = -1,1

def fn(x):
    return np.exp(x)

def area_ex(fn,a,b):
    return integrate.quad(fn,a,b)[0]


mc_area_graph_aprox(fn,a,b,int(1e5))
mc_area_aprox(fn,a,b,int(1e5))

# Resolución de la integral
Con los puntos generados, estime por Monte Carlo las dos integrales que aparecen en el literal c), así:

$$
\begin{gathered}
\int_{-u}^{u} u_{z} e^{\beta B u_{z}} d u_{z} \approx \frac{1}{N} \sum_{i=1}^{N} u_{z, i} e^{\beta B u_{z, i}} \\
Z=\int_{-u}^{u} e^{\beta B u_{z}} d u_{z} \approx \frac{1}{N} \sum_{i=1}^{N} e^{\beta B u_{z, i}}
\end{gathered}
$$



In [ ]:
#@title Constantes del Sistema
#Se deja todo en terminos de K_b
k_b = 1#constants.Boltzmann
T=1
Beta= 1/(T*k_b)
B=10
a,b=-2,2

In [ ]:
#@title Integrales
def arg_z(u_z):
    return np.exp(Beta*B*u_z)

def arg_vp_nonorm(u_z):
    return u_z*np.exp(Beta*B*u_z)

pre = int(1e6)

In [ ]:
#@title Integral función de partición
mc_area_graph_aprox(arg_z,a,b,pre)
z_area = mc_area_aprox(arg_z,a,b,pre)[0]
print(r"El valor de $\left\langle u_{z}\right\rangle$ para $u=2$ es {z_area}")

In [ ]:
mc_area_graph_aprox(arg_vp_nonorm,a,b,pre)
mc_area_aprox(arg_vp_nonorm,a,b,pre)[0]

In [ ]:
vp = mc_area_aprox(arg_vp_nonorm,a,b,pre)[0]/mc_area_aprox(arg_z,a,b,pre)[0]
vp


#Grafica de $\left\langle u_{z}\right\rangle$
Estime a partir de ellas el valor de $\left\langle u_{z}\right\rangle$. Grafique $\left\langle u_{z}\right\rangle$. En función de $x$ para $0.1 \leq x<4.0$ en pasos de 0.1 y compare con la curva teórica que se obtiene en el literal c).[texto del vínculo](https://)

In [ ]:
u_i =0.1
u_f = 4
N_pasos = 4*int(1e1)#40
u = np.linspace(u_i,u_f,N_pasos)
u

In [ ]:
#@title Función de Partición
def z(u):
    return ((4*np.pi)/(u*Beta*B))*np.sinh(Beta*B*u)

def z_num(u):
    a,b=-u,u
    area_z = ((2*np.pi)/u)*mc_area_aprox(arg_z,a,b,pre)
    return area_z[0]

In [ ]:
#@title Gráfica de la Función de Partición
plt.title(r"Función Partición con $B$=10 y $\beta=1$ ")
plt.plot(u,z(u),"r:", markersize = 3, label = 'Valores teóricos')
         #label = r"Función exacta $\frac{2\pi}{u}\sinh(2u)$")
plt.scatter(u,z_num(u), c = 'darkblue', label = 'Valores numericos')
plt.ylabel("Z")
plt.xlabel("u")
plt.grid(True, linestyle='--', linewidth=0.8, alpha=0.9)
plt.legend()
# plt.savefig("FuncionParticion")

In [ ]:
#@title Distribución del valor esperado no normalizado
def p(u):
    return ((4*np.pi)/(u*B*Beta))*(u*np.cosh(u*B*Beta) - (1/(Beta*B))*np.sinh(u*B*Beta))

def p_num(u):
    a,b=-u,u
    area_prob = ((2*np.pi)/u)*mc_area_aprox(arg_vp_nonorm,a,b,pre)
    return area_prob[0]


In [ ]:
#@title Gráfica del valor esperado no normalizado
plt.title(r"Distribución del valor esperado no normalizado con $B$=10 y $\beta=1$")
plt.plot(u,p(u), "r:", markersize = 3, label = 'Valores teóricos')
plt.scatter(u,p_num(u), c = 'darkblue', label = 'Valores numericos')
plt.ylabel("P(u)")
plt.xlabel("u")
plt.grid(True, linestyle='--', linewidth=0.8, alpha=0.9)
plt.legend()
# plt.savefig("Probabilidad")

In [ ]:
1e-1

In [ ]:
#@title Valor esperado
def vp(u):
    return u*(1/np.tanh(u*B*Beta) - 1/(u*B*Beta))

def vp_num(u):
    return(p_num(u))/(z_num(u))


In [ ]:
#@title Gráfica del Valor esperado
plt.title(r"Valor esperado con $B$=10 y $\beta=1$")
plt.plot(u,vp(u),"r:", markersize = 3, label = 'Valores teóricos')
# plt.scatter(u,vp_num(u),c = 'darkblue', label = 'Valores numericos')
plt.ylabel(r"$\left\langle u_{z}\right\rangle$")
plt.xlabel("u")
plt.grid(True, linestyle='--', linewidth=0.8, alpha=0.9)
plt.legend()
# plt.savefig("ValorEsperado")